# Prompt-Sandbox Quickstart Guide

This notebook demonstrates the basic functionality of prompt-sandbox for prompt engineering experiments.

## Installation

```bash
pip install -e .
```

## 1. Create Prompt Templates

Start by defining prompt templates with variables.

In [ ]:
from prompt_sandbox.config.schema import PromptConfig
from prompt_sandbox.prompts.template import PromptTemplate

# Define a simple Q&A prompt
qa_config = PromptConfig(
    name="qa_simple",
    template="Question: {{question}}\nAnswer:",
    variables=["question"]
)

qa_template = PromptTemplate(qa_config)

# Render with actual values
rendered = qa_template.render(question="What is the capital of France?")
print(rendered)

Expected output:
```
Question: What is the capital of France?
Answer:
```

## 2. Compare Different Prompt Styles

Let's create two different prompting styles to compare.

In [ ]:
# Direct prompting
direct_prompt = PromptTemplate(PromptConfig(
    name="direct",
    template="Q: {{question}}\nA:",
    variables=["question"]
))

# Chain-of-thought prompting
cot_prompt = PromptTemplate(PromptConfig(
    name="chain_of_thought",
    template="Q: {{question}}\nLet's think step by step:\nA:",
    variables=["question"]
))

# Compare renderings
test_question = "What is 15% of 80?"

print("=== Direct Prompt ===")
print(direct_prompt.render(question=test_question))

print("\n=== Chain-of-Thought Prompt ===")
print(cot_prompt.render(question=test_question))

## 3. Using Model Backends

Prompt-sandbox supports multiple model backends. Here we'll use a mock backend for demonstration.

In [ ]:
from prompt_sandbox.models.base import ModelBackend, GenerationResult

# For this demo, we'll create a simple mock backend
class SimpleMockBackend(ModelBackend):
    def __init__(self):
        self.model_name = "mock-model"
    
    def generate(self, prompt, **kwargs) -> GenerationResult:
        # Simple rule-based response for demo
        if "capital" in prompt.lower() and "france" in prompt.lower():
            response = "The capital of France is Paris."
        elif "15%" in prompt and "80" in prompt:
            response = "12"
        else:
            response = "I'm a simple mock model for demonstration."
        
        return GenerationResult(
            prompt=prompt,
            generated_text=response,
            tokens_generated=len(response.split()),
            generation_time=0.1,
            model_name=self.model_name
        )

# Test the mock backend
model = SimpleMockBackend()
prompt_text = direct_prompt.render(question="What is the capital of France?")
result = model.generate(prompt_text)

print(f"Prompt: {result.prompt}")
print(f"Response: {result.generated_text}")
print(f"Tokens: {result.tokens_generated}")

## 4. Evaluation Metrics

Evaluate model outputs using various metrics.

In [ ]:
from prompt_sandbox.evaluators import BLEUEvaluator, ROUGEEvaluator

# Create evaluators
bleu = BLEUEvaluator()
rouge = ROUGEEvaluator()

# Test outputs
reference = "Paris"
prediction = "The capital of France is Paris."

# Evaluate
bleu_score = bleu.evaluate(prediction, reference)
rouge_score = rouge.evaluate(prediction, reference)

print(f"BLEU Score: {bleu_score:.4f}")
print(f"ROUGE Score: {rouge_score:.4f}")

print("\nBLEU measures precision of n-grams (higher = better match)")
print("ROUGE measures recall of n-grams (higher = better coverage)")

## 5. Next Steps

Check out these notebooks for more advanced usage:

- `02_full_experiment.ipynb` - Running complete experiments with multiple prompts and models
- `03_visualization.ipynb` - Visualizing and comparing results

Or explore the examples directory:
```bash
python examples/complete_example.py
```